<a href="https://colab.research.google.com/github/PadmarajBhat/Rapids.AI/blob/master/notMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

From : https://www.ritchieng.com/machine-learning/deep-learning/tensorflow/notmnist/

import the downloading the notMNIST modules

In [3]:
class Xyz:
  def abc(self):
    self.aaa = 10

Q = Xyz()
try:
  print(Q.aaa)
except:
  print("aaa not found")
Q.abc()
print(Q.aaa)


aaa not found
10


In [43]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import imageio

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline


class ImageLoader:
  def __init__(self):
    num_classes = 10
    url = 'http://commondatastorage.googleapis.com/books1000/'
    last_percent_reported = None
    num_classes = 10
    np.random.seed(133)
    image_size = 28  # Pixel width and height.
    pixel_depth = 255.0  # Number of levels per pixel.
    
  def download_progress_hook(count, blockSize, totalSize):
    """A hook to report the progress of a download. This is mostly intended for users with
    slow internet connections. Reports every 1% change in download progress.
    """
    global last_percent_reported
    percent = int(count * blockSize * 100 / totalSize)

    if last_percent_reported != percent:
      if percent % 5 == 0:
        sys.stdout.write("%s%%" % percent)
        sys.stdout.flush()
      else:
        sys.stdout.write(".")
        sys.stdout.flush()

      last_percent_reported = percent
    
  def maybe_download(filename, expected_bytes, force=False):
    """Download a file if not present, and make sure it's the right size."""
    if force or not os.path.exists(filename):
      print('Attempting to download:', filename) 
      filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
      print('\nDownload Complete!')
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
      print('Found and verified', filename)
    else:
      raise Exception(
        'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename
  
  def maybe_extract(filename, force=False):
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
    if os.path.isdir(root) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping extraction of %s.' % (root, filename))
    else:
      print('Extracting data for %s. This may take a while. Please wait.' % root)
      tar = tarfile.open(filename)
      sys.stdout.flush()
      tar.extractall()
      tar.close()
    data_folders = [
      os.path.join(root, d) for d in sorted(os.listdir(root))
      if os.path.isdir(os.path.join(root, d))]
    if len(data_folders) != num_classes:
      raise Exception(
        'Expected %d folders, one per class. Found %d instead.' % (
          num_classes, len(data_folders)))
    print(data_folders)
    return data_folders
  
  def load_letter(folder, min_num_images):
    """Load the data for a single letter label."""
    image_files = os.listdir(folder)
    dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                           dtype=np.float32)
    print(folder)
    num_images = 0
    for image in image_files:
      image_file = os.path.join(folder, image)
      try:
        image_data = (imageio.imread(image_file).astype(float) - 
                      pixel_depth / 2) / pixel_depth
        if image_data.shape != (image_size, image_size):
          raise Exception('Unexpected image shape: %s' % str(image_data.shape))
        dataset[num_images, :, :] = image_data
        num_images = num_images + 1
      except (IOError, ValueError) as e:
        print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')

    dataset = dataset[0:num_images, :, :]
    if num_images < min_num_images:
      raise Exception('Many fewer images than expected: %d < %d' %
                      (num_images, min_num_images))

    print('Full dataset tensor:', dataset.shape)
    print('Mean:', np.mean(dataset))
    print('Standard deviation:', np.std(dataset))
    return dataset

  def maybe_pickle(data_folders, min_num_images_per_class, force=False):
    dataset_names = []
    for folder in data_folders:
      set_filename = folder + '.pickle'
      dataset_names.append(set_filename)
      if os.path.exists(set_filename) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping pickling.' % set_filename)
      else:
        print('Pickling %s.' % set_filename)
        dataset = load_letter(folder, min_num_images_per_class)
        try:
          with open(set_filename, 'wb') as f:
            pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
          print('Unable to save data to', set_filename, ':', e)

    return dataset_names

    def call_download_extract():
      train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
      test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)
      
      train_folders = maybe_extract(train_filename)
      test_folders = maybe_extract(test_filename)
      
      self.train_datasets = maybe_pickle(train_folders, 45000)
      self.test_datasets = maybe_pickle(test_folders, 1800)
  
abcd = ImageLoader()


7
10
http://commondatastorage.googleapis.com/books1000/
10


AttributeError: ignored

### Extract them

In [3]:
Image(filename="notMNIST_small/A/Q0NXaWxkV29yZHMtQm9sZEl0YWxpYy50dGY=.png")

In [4]:
os.listdir("notMNIST_small/")

['D', 'J', 'A', 'B', 'C', 'G', 'I', 'H', 'F', 'E']

How to display multiple Images in a cell?
https://stackoverflow.com/a/49487396/8693106

In [5]:
import random
listOfFiles = []
for (dirpath, dirnames, filenames) in os.walk("notMNIST_small/"):
  listOfFiles += [os.path.join(dirpath, file) for file in filenames]
  
  for f in listOfFiles:
    if random.randint(0,1000) == random.randint(0,1000):
      print(f)
      display(Image(filename=f))
      #break
  #break

notMNIST_small/D/QWNhbnRodXMgU1NpIEl0YWxpYy50dGY=.png


notMNIST_small/D/Q29ydGluLnR0Zg==.png


notMNIST_small/D/Q2FuZHkgU3F1YXJlIEJUTiBDb25kLnR0Zg==.png


notMNIST_small/D/RW5zY2hlZGUtTWVkaXVtLm90Zg==.png


notMNIST_small/J/RnJhbmtsaW5Hb3RoaWMtTWVkQ25kLm90Zg==.png


...

notMNIST_small/C/Q3Jvbm9zUHJvLUNhcHRJdC5vdGY=.png


notMNIST_small/C/SVRDQXZhbnRHYXJkZVN0ZC1Cb2xkQ24ub3Rm.png


notMNIST_small/I/SGVsdmV0aWNhLU5hcnJvdy1Cb2xkLm90Zg==.png


notMNIST_small/H/QmF1aGF1cyBCb2xkLnR0Zg==.png


notMNIST_small/F/QmFjY3VzIFJlZ3VsYXIudHRm.png


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [22]:
!ls -l "/content/drive/My Drive/Colab Notebooks/notMNiST/notMNIST_large"

total 648171
-rw------- 1 root root 165932198 Jul 26 23:36 G.pickle
-rw------- 1 root root 165932198 Jul 26 23:37 H.pickle
-rw------- 1 root root 165932198 Jul 26 23:40 I.pickle
-rw------- 1 root root 165929062 Jul 26 23:45 J.pickle


In [25]:
!cp notMNIST_small/*.pickle "/content/drive/My Drive/Colab Notebooks/notMNiST/notMNIST_small"
!ls -l "/content/drive/My Drive/Colab Notebooks/notMNiST/notMNIST_small"

total 57347
-rw------- 1 root root 5870749 Jul 26 23:47 A.pickle
-rw------- 1 root root 5873885 Jul 26 23:47 B.pickle
-rw------- 1 root root 5873885 Jul 26 23:47 C.pickle
-rw------- 1 root root 5873885 Jul 26 23:47 D.pickle
-rw------- 1 root root 5873885 Jul 26 23:47 E.pickle
-rw------- 1 root root 5870749 Jul 26 23:47 F.pickle
-rw------- 1 root root 5870749 Jul 26 23:47 G.pickle
-rw------- 1 root root 5870749 Jul 26 23:47 H.pickle
-rw------- 1 root root 5870749 Jul 26 23:47 I.pickle
-rw------- 1 root root 5870749 Jul 26 23:47 J.pickle


In [26]:
!cp notMNIST_large/*.pickle "/content/drive/My Drive/Colab Notebooks/notMNiST/notMNIST_large"
!ls -l "/content/drive/My Drive/Colab Notebooks/notMNiST/notMNIST_large"

total 1620417
-rw------- 1 root root 165922781 Jul 26 23:49 A.pickle
-rw------- 1 root root 165929053 Jul 26 23:49 B.pickle
-rw------- 1 root root 165932189 Jul 26 23:49 C.pickle
-rw------- 1 root root 165929053 Jul 26 23:49 D.pickle
-rw------- 1 root root 165932189 Jul 26 23:49 E.pickle
-rw------- 1 root root 165932189 Jul 26 23:49 F.pickle
-rw------- 1 root root 165932189 Jul 26 23:49 G.pickle
-rw------- 1 root root 165932189 Jul 26 23:49 H.pickle
-rw------- 1 root root 165932189 Jul 26 23:49 I.pickle
-rw------- 1 root root 165929053 Jul 26 23:49 J.pickle


Some Useful Commands:

```
!echo $PATH
import os
os.environ['PATH'] = ":/usr/local:" +os.environ['PATH']
!echo $PATH

Here /usr/local is set because Anaconda software is installed at /usr/local
However, as anaconda has set some variable with which colab is able to access anaconda directory. It is not PATH; may be PYTHONPATH.
```

In [12]:
!ls -l notMNIST_large/*.pickle

-rw-r--r-- 1 root root 165922781 Jul 26 23:16 notMNIST_large/A.pickle
-rw-r--r-- 1 root root 165929053 Jul 26 23:17 notMNIST_large/B.pickle
-rw-r--r-- 1 root root 165932189 Jul 26 23:17 notMNIST_large/C.pickle
-rw-r--r-- 1 root root 165929053 Jul 26 23:17 notMNIST_large/D.pickle
-rw-r--r-- 1 root root 165932189 Jul 26 23:18 notMNIST_large/E.pickle
-rw-r--r-- 1 root root 165932189 Jul 26 23:18 notMNIST_large/F.pickle
-rw-r--r-- 1 root root 165932189 Jul 26 23:18 notMNIST_large/G.pickle
-rw-r--r-- 1 root root 165932189 Jul 26 23:19 notMNIST_large/H.pickle
-rw-r--r-- 1 root root 165932189 Jul 26 23:19 notMNIST_large/I.pickle
-rw-r--r-- 1 root root 165929053 Jul 26 23:19 notMNIST_large/J.pickle


In [7]:
import pynvml


pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)

if device_name != b'Tesla T4':
  raise Exception("""
    Unfortunately this instance does not have a T4 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.

    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 2))



Exception: ignored

In [0]:
!bash -c "$(wget -q https://s3.amazonaws.com/blazingsql-colab/install.sh -O -)"

In [0]:
!cat /blazingsql/demo1.py
!python /blazingsql/demo1.py

In [0]:
import cudf

In [0]:
def make_arrays(nb_rows, img_size):
  if nb_rows:
    dataset = np.ndarray((nb_rows, img_size, img_size), dtype=np.float32)
    labels = np.ndarray(nb_rows, dtype=np.int32)
  else:
    dataset, labels = None, None
  return dataset, labels

def merge_datasets(pickle_files, train_size, valid_size=0):
  num_classes = len(pickle_files)
  valid_dataset, valid_labels = make_arrays(valid_size, image_size)
  train_dataset, train_labels = make_arrays(train_size, image_size)
  vsize_per_class = valid_size // num_classes
  tsize_per_class = train_size // num_classes
    
  start_v, start_t = 0, 0
  end_v, end_t = vsize_per_class, tsize_per_class
  end_l = vsize_per_class+tsize_per_class
  for label, pickle_file in enumerate(pickle_files):       
    try:
      with open(pickle_file, 'rb') as f:
        letter_set = pickle.load(f)
        # let's shuffle the letters to have random validation and training set
        np.random.shuffle(letter_set)
        if valid_dataset is not None:
          valid_letter = letter_set[:vsize_per_class, :, :]
          valid_dataset[start_v:end_v, :, :] = valid_letter
          valid_labels[start_v:end_v] = label
          start_v += vsize_per_class
          end_v += vsize_per_class
                    
        train_letter = letter_set[vsize_per_class:end_l, :, :]
        train_dataset[start_t:end_t, :, :] = train_letter
        train_labels[start_t:end_t] = label
        start_t += tsize_per_class
        end_t += tsize_per_class
    except Exception as e:
      print('Unable to process data from', pickle_file, ':', e)
      raise
    
  return valid_dataset, valid_labels, train_dataset, train_labels
            
            
train_size = 200000
valid_size = 10000
test_size = 10000

valid_dataset, valid_labels, train_dataset, train_labels = merge_datasets(
  train_datasets, train_size, valid_size)
_, _, test_dataset, test_labels = merge_datasets(test_datasets, test_size)

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

This was the traditional approach where local variable train_dataset was nd array and was trying to load all of the training data set in the memory. This is definitely not scalable.

We have seen in the rapid intro video that the read csv are delayed function and are read each time when db operations are done. I guess this read each time makes is compliant to ACID property. 

Similarly, we need to create the dataframe from the right when we read the image in our case. Approach I m thinking of is:
* Read the image
* convert the nxn to n*n ==> 28x28 to 786 array
* Store it as cudf series of n*n and label
* append the cudf series to a df
* split the df to training and validation set
* use the training set for training various rapid algo
* of course use the validation set to verify those models
* test_folders(notMNIST/small/*) will be testing dataset
* repeat the exercise with BlazingDB operation 
* note that time at each stage for comparison
* may be we can also see some visualization in rapids regarding data access intervals and computation cost

In [0]:
train_folders, test_folders